In [116]:
# AV ETY Simulation v9: Added Loops, introduced realistic parameters + VRS, vessel assignment from passive vessels, first created (=alphabetical)
import sys
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install salabim


In [117]:
# Import required functions and libraries

import random
import pandas as pd
import salabim as sim

In [118]:
class CargoGenerator(sim.Component):
    def process(self):
        while True: # Continuous loop
            Cargo(load_port=self.load_port, discharge_port=random.choice(self.discharge_ports))
            yield self.hold(sim.Normal(mean=self.cargo_interval_mean, standard_deviation=self.cargo_interval_std).sample())

    def setup(self, load_port, parameters):
        self.load_port = load_port
        self.cargo_interval_mean = parameters["cargo_interval_mean"]
        self.cargo_interval_std = parameters["cargo_interval_std"]
        self.discharge_ports = parameters["discharge_ports"]


class Cargo(sim.Component):
    def process(self):
        self.enter(cargo_queue)
        for vessel in vessels:
            if vessel.ispassive():
                vessel.activate()
                break # activate 1 vessel only
        yield self.passivate()

    def setup(self, load_port, discharge_port):
        self.load_port = load_port
        self.discharge_port = discharge_port


class Vessel(sim.Component):
    def process(self):
        while True: # Continuous loop
            while len(cargo_queue) == 0:
                yield self.passivate()
            self.cargo = cargo_queue.pop()
            yield from self.execute_voyage()

    def setup(self):
        self.location = "Antwerp" #All vessels originate in Antwerp

    def execute_voyage(self):
        yield from self.ballast_leg()
        yield from self.load_cargo()
        yield from self.laden_leg()
        yield from self.discharge_cargo()

    def load_cargo(self):
        port_time = LOAD_DISCHARGE_TIME + sim.Normal(MEAN_PORT_DISRUPT, 0.25)
        yield self.hold(port_time)

    def discharge_cargo(self):
        self.location = self.cargo.discharge_port
        port_time = LOAD_DISCHARGE_TIME + sim.Normal(MEAN_PORT_DISRUPT, 0.25)
        yield self.hold(port_time)
        self.cargo.activate()
        print("{:.2f} - {}: {} discharged in {} \n".format(env.now(), self.name(), self.cargo.name(), self.location))

    def laden_leg(self):
        distance = DISTANCES.loc[self.cargo.load_port, self.cargo.discharge_port]
        travel_time_laden = distance / CRUISE_SPEED / 24 * ( 1 + VRS / 100 )
        print("{:.2f} - {}: {} loaded at {}, Departure laden to {} \n".format(env.now(), self.name(), self.cargo.name(),
                                                                              self.cargo.load_port,
                                                                              self.cargo.discharge_port))
        yield self.hold(travel_time_laden)
        print(
            "{:.2f} - {}: Arrive laden at {} (Distance: {} NM, Travel time: {:.2f} days) \n".format(env.now(), self.name(),
                                                                                                 self.cargo.discharge_port,
                                                                                                 distance,
                                                                                                 travel_time_laden))

    def ballast_leg(self):
        ballast_distance = DISTANCES.loc[self.location, self.cargo.load_port]
        travel_time_ballast = ballast_distance / CRUISE_SPEED / 24 * ( 1 + VRS / 100 )
        print("{:.2f} - {}: Departure ballast to {} \n".format(env.now(), self.name(), self.cargo.load_port))
        yield self.hold(travel_time_ballast)
        print("{:.2f} - {}: Arrive ballast at {} (Distance: {} NM, Travel time: {:.2f} days) \n".format(env.now(),
                                                                                                     self.name(),
                                                                                                     self.cargo.load_port,
                                                                                                     ballast_distance,
                                                                                                     travel_time_ballast))

def average_passive_time(components: list):
    total_time = 0
    no_components = len(components)
    for component in components:
        total_time += component.status.value_duration("passive")
    return total_time / no_components / SIM_DURATION

In [119]:
# Define variable parameters
SIM_DURATION = 365 # in days
LOAD_DISCHARGE_TIME = 1.25 # in days
CRUISE_SPEED = 15 # knots
VRS = 8 # Percent

NO_OF_VESSELS = [6] # To loop through options, add as follows: [X, Y, Z]
BFB_CARGOES = [150] # Per year (To loop through options, add as follows: [X, Y, Z])
MEAN_PORT_DISRUPT_VALUES = [0.25] # In days (To loop through options, add as follows: [X, Y, Z])
CRUISE_SPEEDS = [15] # knots (To loop through options, add as follows: [X, Y, Z])


# Define constant parameters

def load_ports(bfb_cargoes):
    return {
        "Braefoot Bay": {
            "cargo_interval_mean": 365/bfb_cargoes,
            "cargo_interval_std": 0.5,
            "discharge_ports": ["Antwerp"]
            },
        "Porvoo": {
            "cargo_interval_mean": 365/20,
            "cargo_interval_std": 3,
            "discharge_ports": ["Stenungsund", "Stenungsund", "Stade", "Stade", "Antwerp"]
        },
        "Rafnes": {
            "cargo_interval_mean": 365/78,
            "cargo_interval_std": 1,
            "discharge_ports": ["Antwerp", "Antwerp", "Antwerp", "Antwerp", "Antwerp", "Antwerp", "Stenungsund", "Stenungsund", "Stenungsund", "Stenungsund", "Stenungsund", "Stade", "Stade"]
        },
        "Le Havre": {
            "cargo_interval_mean": 365/9,
            "cargo_interval_std": 5,
            "discharge_ports": ["Antwerp"]
        },
    }

DISTANCES = pd.DataFrame([
    [0, 435, 247, 1234, 572, 368, 570],
    [435, 0, 503, 1182, 486, 480, 517],
    [247, 503, 0, 1352, 687, 490, 688],
    [1234, 1182, 1352, 0, 796, 1047, 725],
    [572, 586, 687, 796, 0, 385, 112],
    [368, 480, 490, 1047, 385, 0, 383],
    [570, 517, 688, 725, 112, 383, 0]
    ],
    index=["Antwerp", "Braefoot Bay", "Le Havre", "Porvoo", "Rafnes", "Stade", "Stenungsund"],
    columns=["Antwerp", "Braefoot Bay", "Le Havre", "Porvoo", "Rafnes", "Stade", "Stenungsund"]
)

VESSELS = ["Coral Lophelia", "Coral Parensis", "Coral Patula", "Coral Pearl", "Coral Alameda", "Coral Orinda", "Coral Shasta", "Coral Furcata", "Coral Fraseri", "Coral Fungia"]

cargogenerators = {}


In [120]:
if __name__ == "__main__":

    for no_cargoes in BFB_CARGOES:
        LOAD_PORTS = load_ports(no_cargoes)

        for no_vessels in NO_OF_VESSELS:

            for CRUISE_SPEED in CRUISE_SPEEDS:

                for MEAN_PORT_DISRUPT in MEAN_PORT_DISRUPT_VALUES:

                        env = sim.Environment(time_unit="days")
                        env.time_to_str_format('{:10.2f}')

                        vessels = [Vessel(name=name) for name in VESSELS[:no_vessels]]

                        for load_port, parameters in LOAD_PORTS.items():
                            cargogenerators[load_port] = CargoGenerator(load_port=load_port, parameters=parameters)

                        cargo_queue = sim.Queue('cargo_queue')

                        env.run(till=SIM_DURATION)

                        cargo_queue.print_statistics(as_str=True)
                        cargo_queue.print_histograms(as_str=True)
                        cargo_queue.print_info(as_str=True)

                        for vessel in vessels:
                            print(vessel)
                            print(vessel.status.print_histogram(values=True, as_str=True))

                        print("Average idle time: {:.2%}".format(average_passive_time(vessels)))

0.00 - Coral Lophelia: Departure ballast to Braefoot Bay 

0.00 - Coral Parensis: Departure ballast to Porvoo 

0.00 - Coral Patula: Departure ballast to Rafnes 

0.00 - Coral Pearl: Departure ballast to Le Havre 

0.74 - Coral Pearl: Arrive ballast at Le Havre (Distance: 247 NM, Travel time: 0.74 days) 

1.30 - Coral Lophelia: Arrive ballast at Braefoot Bay (Distance: 435 NM, Travel time: 1.30 days) 

1.72 - Coral Patula: Arrive ballast at Rafnes (Distance: 572 NM, Travel time: 1.72 days) 

1.89 - Coral Alameda: Departure ballast to Braefoot Bay 

2.27 - Coral Pearl: cargo.3 loaded at Le Havre, Departure laden to Antwerp 

2.68 - Coral Lophelia: cargo.0 loaded at Braefoot Bay, Departure laden to Antwerp 

3.01 - Coral Pearl: Arrive laden at Antwerp (Distance: 247 NM, Travel time: 0.74 days) 

3.11 - Coral Orinda: Departure ballast to Rafnes 

3.20 - Coral Alameda: Arrive ballast at Braefoot Bay (Distance: 435 NM, Travel time: 1.30 days) 

3.49 - Coral Patula: cargo.2 loaded at Rafnes,